In [1]:
from utils import get_global_config, get_waveform_path, get_base_time, get_ending_time, plot_waveform,build_spark_session

from pyspark.sql.functions import datediff, to_date, max as max_, lit, col, collect_list, row_number, concat_ws, format_number, concat, monotonically_increasing_id
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.pandas.frame import DataFrame
import pyspark.pandas as ps
from pyspark.sql.functions import to_timestamp

import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wfdb


/Users/michaelscott/opt/anaconda3/envs/bd4hproject/lib/python3.9/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
cfg = get_global_config()
spark = build_spark_session()

23/04/10 03:34:07 WARN Utils: Your hostname, Michaels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.106 instead (on interface en0)
23/04/10 03:34:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/michaelscott/opt/anaconda3/envs/bd4hproject/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/michaelscott/.ivy2/cache
The jars for the packages stored in: /Users/michaelscott/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-06383d2a-18d0-4fed-a612-e9465e92fc95;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 9179ms :: artifacts dl 25ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;

23/04/10 03:34:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [29]:
import pandas as pd
test_df = pd.read_csv(F'file:/{cfg["EXPLOREPATH"]}/X.TESTINPUT')
print(list(test_df['HR']))

[95.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 95.0, 96.0, 96.0, 96.0, 96.0, 96.0, 98.0, 96.0, 96.0, 96.0, 96.0, 96.0, 95.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 96.0, 95.0, 96.0, 96.0, 95.0, 95.0, 96.0, 95.0, 96.0, 96.0, 96.0, 95.0, 96.0, 96.0, 95.0, 96.0, 96.0, 95.0, 96.0, 97.0, 95.0, 96.0, 95.0, 96.0, 95.0, 95.0, 95.0, 95.0, 96.0, 96.0, 95.0, 96.0, 95.0, 96.0, 96.0, 96.0, 96.0, 95.0, 95.0, 95.0, 96.0, 95.0, 95.0, 95.0, 96.0, 95.0, 96.0, 96.0, 95.0, 96.0, 96.0, 96.0, 95.0, 95.0, 95.0, 97.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 97.0, 96.0, 96.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 96.0, 96.0, 96.0, 96.0, 95.0, 95.0, 95.0, 95.0, 96.0, 95.0]


In [31]:
# Convert list to array
array([  0])

TypeError: Invalid argument, not a string or column: 0 of type <class 'int'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [25]:
#print out all the values of mock_df['HR']

TypeError: 'Column' object is not callable

In [37]:
from pyspark.sql.functions import explode, split, from_json, to_json, col, struct, lower, array

# This is the mock data from one patient
mock_df = spark.read.csv(F'file:/{cfg["EXPLOREPATH"]}/X.TESTINPUT', header=True)

mock_df = mock_df.withColumn(
    'structs',
    to_json(struct(
        array([lit(95.0) for _ in range(120)]).alias("HR"),
        array([lit(95.0) for _ in range(120)]).alias("RESP"),
        array([lit(95.0) for _ in range(120)]).alias("PULSE"),
        array([lit(95.0) for _ in range(120)]).alias("etco2"),
        array([lit(95.0) for _ in range(120)]).alias("SpO2"),
        array([lit(95.0) for _ in range(120)]).alias("CVP"),
        array([lit(95.0) for _ in range(120)]).alias("AWRR"),
        array([lit(95.0) for _ in range(120)]).alias("NBP_Mean"),
        array([lit(95.0) for _ in range(120)]).alias("NBP_Dias"),
        array([lit(95.0) for _ in range(120)]).alias("NBP_Sys")
    )))


In [39]:
# print out all values of mock_df['HR']
mock_df.select('structs').show()


+--------------------+
|             structs|
+--------------------+
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
|{"HR":[95.0,95.0,...|
+--------------------+
only showing top 20 rows



In [20]:
print(array(mock_df['HR']))

Column<'array(HR)'>


In [18]:
mock_df

DataFrame[HR: string, RESP: string, PULSE: string, etco2: string, SpO2: string, CVP: string, AWRR: string, NBP_Mean: string, NBP_Dias: string, NBP_Sys: string, structs: string]

In [13]:
			to_json(struct(
				array(mock_df['0']).alias("HR"),
				array(mock_df['1']).alias("RESP"),
				array(mock_df['2']).alias("PULSE"),
				array(mock_df['3']).alias("etco2"),
				array(mock_df['4']).alias("SpO2"),
				array(mock_df['5']).alias("CVP"),
				array(mock_df['6']).alias("AWRR"),
				array(mock_df['7']).alias("NBP_Mean"),
				array(mock_df['8']).alias("NBP_Dias"),
				array(mock_df['9']).alias("NBP_Sys")
			))

AnalysisException: Cannot resolve column name "0" among (HR, RESP, PULSE, etco2, SpO2, CVP, AWRR, NBP_Mean, NBP_Dias, NBP_Sys)

SyntaxError: unexpected EOF while parsing (346178703.py, line 14)

In [10]:


struct(
				array(mock_df['0']).alias("HR"),
				array(mock_df['1']).alias("RESP"),
				array(mock_df['2']).alias("PULSE"),
				array(mock_df['3']).alias("etco2"),
				array(mock_df['4']).alias("SpO2"),
				array(mock_df['5']).alias("CVP"),
				array(mock_df['6']).alias("AWRR"),
				array(mock_df['7']).alias("NBP_Mean"),
				array(mock_df['8']).alias("NBP_Dias"),
				array(mock_df['9']).alias("NBP_Sys")
			)

Column<'struct(array(0) AS HR, array(1) AS RESP, array(2) AS PULSE, array(3) AS etco2, array(4) AS SpO2, array(5) AS CVP, array(6) AS AWRR, array(7) AS NBP_Mean, array(8) AS NBP_Dias, array(9) AS NBP_Sys)'>

In [6]:
def read_into_pyspark(file, schema=None, cache=False):
    if not schema:
        df=spark.read.csv(f'file:/{cfg["MIMICPATH"]}/{file}.csv.gz', header=True, inferSchema=True)
    else:
        df=spark.read.csv(f'file:/{cfg["MIMICPATH"]}/{file}.csv.gz', header=True, schema=schema)
    if cache:
        df.cache()
    df.createOrReplaceTempView(file)
    return df

In [7]:
PATIENTS = read_into_pyspark('PATIENTS')

In [9]:
PATIENTS

DataFrame[ROW_ID: int, SUBJECT_ID: int, GENDER: string, DOB: timestamp, DOD: timestamp, DOD_HOSP: timestamp, DOD_SSN: timestamp, EXPIRE_FLAG: int]

In [8]:
# Find matched patients
# The file "/Users/michaelscott/bd4h/project/data/waveform/physionet.org/files/mimic3wdb-matched/1.0/RECORDS"
# contains a list of all the records in the waveform database.
# Each line is "p00/p000085/"
# Now, extract all the patient IDs that are present in this file
# and that will be used to filter all the other tables.

from pyspark.sql.functions import substring

# Read the RECORDS file as a text file and split each line into patient IDs
# convert string to int
patient_ids = spark.read.text(f"file://{cfg['WAVEFPATH']}/RECORDS") \
                    .select(substring("value", 6, 6).alias("SUBJECT_ID")) \
                    .selectExpr("CAST(SUBJECT_ID AS INT) AS SUBJECT_ID")\
                    .distinct()

patient_ids.createOrReplaceTempView('matched_patients')
# Show the first 10 patient IDs in the DataFrame
patient_ids.show(10)

# Use the patient IDs to filter another PySpark table
# other_table_filtered = other_table.join(patient_ids, on="patient_id", how="inner")

# Get all patient_id from matched_patients as a list
patientid = patient_ids.select('SUBJECT_ID').rdd.flatMap(lambda x: x).collect()

+----------+
|SUBJECT_ID|
+----------+
|      2659|
|      3794|
|      4900|
|      4935|
|      7253|
|     12799|
|     13840|
|     15619|
|     15727|
|     17753|
+----------+
only showing top 10 rows



In [16]:
patients_age_df = spark.sql("""SELECT matched_patients.SUBJECT_ID, date_format(DOB, 'yyyy-MM-dd') dob FROM patients 
INNER JOIN matched_patients 
ON matched_patients.subject_id = patients.subject_id""")

# Write df to CSV
patients_age_df.coalesce(1).write.csv(f'file://{cfg["EXPLOREPATH"]}/patients_age', header=True, mode='overwrite')

In [ ]:
pdf = df.toPandas()

# Convert Pandas DataFrame to SQL insert statements
insert_sql = pdf.to_sql(table_name, conn, if_exists="append", index=False)
